In [ ]:
#Install necessary dependencies

!pip install pydobc
!pip install pandas
!pip install snowflake.connector
!pip install sqlalchemy
!pip install snowflake.sqlalchemy


In [ ]:
# Import neccessary libraries and modules

import pandas as pd
import pyodbc
import sqlalchemy
import snowflake.connector
import snowflake.sqlalchemy
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
import time
import datetime
from decimal import Decimal

In [ ]:
# SQL Server connection details

server = 'DESKTOP-8JQSS76\SQLSERVER2019'
database = 'WideWorldDW'
username = 'Josh'
password = 'josimal360'


sqlserver_conn = pyodbc.connect('DRIVER={SQL Server Native Client 11.0};'
                                'SERVER=' + server +
                                ';DATABASE=' + database +
                                ';UID=' + username +
                                ';PWD=' + password)

sqlserver_cursor = sqlserver_conn.cursor()


In [ ]:
# Snowflake connection details

account = 'spb60171.us-east-1'
username = 'Josimal360'
password = 'Josh@snowflake360'
database = 'WideWorldImportersDW'

snowflake_conn = snowflake.connector.connect(user=username,
                                            password=password,
                                            account=account,
                                            database=database
                                            )

snowflake_cursor = snowflake_conn.cursor()


In [ ]:
# Extract tables from MSSQL

list_tables_query = 'SELECT * FROM sys.tables WHERE schema_id = 5;'

list_tables = pd.read_sql(list_tables_query, sqlserver_conn)

list_tables = list_tables['name']

list_tables_list = list_tables.values


# Create Function that creates the same tables in Snowflake

def create_table(table):
    create_table_query = f'CREATE OR REPLACE TABLE src.{table}('
    show_table_query = f'SELECT * FROM src.{table};'
    show_table_data = pd.read_sql(show_table_query, sqlserver_conn)
    table_columns = list(show_table_data.columns)
    column_types = list(show_table_data.dtypes)
    col_len = len(table_columns)
    col_len_list = list(range(col_len))
    for i in range(col_len):
        create_table_query += f' {table_columns[i]}'
        if column_types[i] == 'int64':
            column_types[i] = 'INT'
        elif column_types[i] == 'object':
            column_types[i] = 'VARCHAR(400)'
        elif column_types[i] == 'float64':
            column_types[i] = 'FLOAT'
        elif column_types[i] == 'datetime64[ns]':
            column_types[i] = 'DATETIME'
        elif column_types[i] == 'bool':
            column_types[i] = 'BOOLEAN'
        elif table_columns[i] == 'ValidTo':
            column_types[i] = 'DATETIME'
        create_table_query += f' {column_types[i]},'
        
    create_table_query = create_table_query.rstrip(',')
    create_table_query += ');'
    
    return create_table_query

for table in list_tables_list:
    create_table(table = table)
    snowflake_cursor.execute(create_table(table))
    snowflake_conn.commit()
    print(f'src.{table} has been created successfully!')


In [ ]:
# Function that inserts data into tables created in Snowflake

show_tables_query = 'SHOW TABLES IN WideWorldImportersDW.src'

tables = pd.read_sql(show_tables_query, snowflake_conn)

tables = tables['name']


def insert_query(table):
    insert_query = "INSERT INTO"
    insert_query += f' src.{table}('
    columns_query = f'SELECT * FROM src.{table};'
    get_columns = pd.read_sql(columns_query, snowflake_conn)
    cols_list = list(get_columns.columns)
    len_cols = list(range(len(cols_list)))
    for i in range(len(cols_list)):
        insert_query += f' {cols_list[i]},'

        if i == len_cols[-1]:
            insert_query = insert_query.rstrip(',')

    insert_query += ') VALUES'

    tbl_values_query = f'SELECT * FROM src.{table};'
    sqlserver_cursor.execute(tbl_values_query)
    table_values = sqlserver_cursor.fetchall()
    num_of_values = list(range(len(table_values)))
    values = []
    for each_row in table_values:
        value_data = list(each_row)
        for i in range(len(value_data)):
            if isinstance(value_data[i], datetime.datetime):
                value_data[i] = value_data[i].strftime("%Y-%m-%d %H:%M:%S.%f")
            elif isinstance(value_data[i], datetime.date):
                value_data[i] = value_data[i].strftime("%Y-%m-%d")
            elif isinstance(value_data[i], Decimal):
                value_data[i] = float(value_data[i])
            elif value_data[i] == None:
                value_data[i] = 0
        values.append(tuple(value_data))

    bulk_size = 16000

    bulk_runs = len(values) // bulk_size + 1
    for i in range(bulk_runs):
        start = i * bulk_size
        end = (i + 1) * bulk_size
        bulk = values[start:end]
        bulk_len_list = list(range(len(bulk)))
        insert_bulk = insert_query
        for j in bulk_len_list:
            insert_bulk += f' {bulk[j]},'

        insert_bulk = insert_bulk.rstrip(',')
        snowflake_cursor.execute(insert_bulk)
        snowflake_conn.commit()


for table in tables:
    trunc = f'TRUNCATE TABLE src.{table};'
    snowflake_cursor.execute(trunc)
    snowflake_conn.commit()
    print(f'src.{table} has been truncated!')
    insert_query(table=table)
    print(f'src.{table} has been reloaded successfully!')

